<a href="https://colab.research.google.com/github/3132david/fooder/blob/main/Fooder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import ipywidgets as widgets
from IPython.display import display, clear_output
from PIL import Image
import requests
from pyzbar import pyzbar
import cv2
import numpy as np
from google.colab import files
import io

class BarcodeNutritionApp:
    def __init__(self):
        # --- WIDGETS ---
        self.upload_btn = widgets.Button(description="📷 Upload Barcode Image", button_style='primary')
        self.output = widgets.Output()
        self.status_label = widgets.Label(value="Select an image to get started")
        self.progress = widgets.IntProgress(value=0, min=0, max=10, step=1, description='Loading:', bar_style='info')

        # --- LAYOUT ---
        vbox_layout = widgets.Layout(display='flex',
                                     flex_flow='column',
                                     align_items='center',
                                     width='100%')
        self.app_layout = widgets.VBox([
            widgets.HTML("<h1>🍎 Barcode Nutrition Scanner</h1>"),
            self.upload_btn,
            self.output,
            self.progress,
            self.status_label
        ], layout=vbox_layout)

        # --- EVENT HANDLERS ---
        self.upload_btn.on_click(self.on_upload_clicked)

    def on_upload_clicked(self, b):
        uploaded = files.upload()
        if uploaded:
            # Get the first uploaded file
            filename = next(iter(uploaded))
            content = uploaded[filename]
            self.scan_barcode(content, filename)

    def scan_barcode(self, file_content, filename):
        try:
            # Read image with OpenCV from memory
            image = cv2.imdecode(np.frombuffer(file_content, np.uint8), cv2.IMREAD_COLOR)
            if image is None:
                raise Exception("Could not read image file")

            # Decode barcodes
            barcodes = pyzbar.decode(image)

            if not barcodes:
                self.status_label.value = "No barcode found in image"
                self.progress.value = 0
                with self.output:
                    clear_output(wait=True)
                    print("No barcode detected. Please try another image.")
                return

            # Get the first barcode
            barcode = barcodes[0]
            barcode_data = barcode.data.decode('utf-8')
            print(f"Decoded Barcode: {barcode_data}")

            with self.output:
                clear_output(wait=True)
                self.status_label.value = "Scanning barcode..."
                self.progress.value = 2

                self.status_label.value = f"Found barcode: {barcode_data}. Getting nutrition info..."
                self.progress.value = 5

                # Display the uploaded image
                img = Image.open(io.BytesIO(file_content))
                img.thumbnail((300, 200))
                display(img)

                # Get nutrition info
                self.get_nutrition_info(barcode_data)

        except Exception as e:
            self.progress.value = 0
            self.status_label.value = "Error scanning barcode"
            with self.output:
                clear_output(wait=True)
                print(f"Error scanning barcode: {str(e)}")

    def get_nutrition_info(self, barcode):
        self.progress.value = 7
        try:
            # Call Open Food Facts API
            url = f"https://world.openfoodfacts.org/api/v0/product/{barcode}.json"
            response = requests.get(url, timeout=10)
            data = response.json()

            self.progress.value = 10

            if data['status'] == 1:  # Product found
                product = data['product']
                self.display_nutrition_info(product)
                self.status_label.value = "Nutrition information loaded successfully!"
            else:
                self.status_label.value = "Product not found in database"
                with self.output:
                    clear_output(wait=True)
                    print("Product not found in the nutrition database.")

        except requests.exceptions.RequestException:
            self.progress.value = 0
            self.status_label.value = "Network error - check internet connection"
            with self.output:
                clear_output(wait=True)
                print("Could not connect to nutrition database. Check your internet connection.")
        except Exception as e:
            self.progress.value = 0
            self.status_label.value = "Error getting nutrition info"
            with self.output:
                clear_output(wait=True)
                print(f"Error getting nutrition info: {str(e)}")

    def display_nutrition_info(self, product):
        with self.output:
            clear_output(wait=True)

            # Product name
            name = product.get('product_name', 'Unknown Product')
            display(widgets.HTML(f"<h2>📦 {name}</h2>"))

            # Create info cards
            self.create_info_card("🔥 Calories", self.get_calories(product), "#e74c3c")
            self.create_info_card("📊 Health Grade", self.get_health_grade(product), "#27ae60")
            self.create_info_card("🍯 Sugar", self.get_nutrient(product, 'sugars'), "#f39c12")
            self.create_info_card("🧂 Salt", self.get_nutrient(product, 'salt'), "#9b59b6")

            # Ingredients (if available)
            ingredients = product.get('ingredients_text', '')
            if ingredients:
                display(widgets.HTML("<h3>🧾 Ingredients</h3>"))
                display(widgets.HTML(f"<p>{ingredients}</p>"))

    def create_info_card(self, title, value, color):
        card = widgets.HBox([
            widgets.Label(value=title, layout=widgets.Layout(width='100px')),
            widgets.Label(value=value)
        ], layout=widgets.Layout(
            border=f'2px solid {color}',
            padding='5px',
            margin='2px',
            width='300px'
        ))
        display(card)


    def get_calories(self, product):
        """Extract calories per 100g"""
        nutriments = product.get('nutriments', {})
        calories = nutriments.get('energy-kcal_100g') or nutriments.get('energy_100g')

        if calories:
            return f"{calories} kcal/100g"
        return "Not available"

    def get_health_grade(self, product):
        """Get Nutri-Score grade (A-E)"""
        grade = product.get('nutrition_grades', '').upper()
        if grade:
            grade_colors = {'A': '🟢', 'B': '🟡', 'C': '🟠', 'D': '🔴', 'E': '🔴'}
            emoji = grade_colors.get(grade, '⚪')
            return f"{emoji} Grade {grade}"
        return "Not rated"

    def get_nutrient(self, product, nutrient_name):
        """Get specific nutrient value per 100g"""
        nutriments = product.get('nutriments', {})
        value = nutriments.get(f'{nutrient_name}_100g')

        if value is not None:
            unit = 'g' if nutrient_name != 'sodium' else 'mg'
            return f"{value}{unit}/100g"
        return "Not available"

def main():
    app = BarcodeNutritionApp()
    display(app.app_layout)

if __name__ == "__main__":
    main()

Saving nutella_bc2.jpg to nutella_bc2 (4).jpg
Decoded Barcode: 3017620425035


In [2]:
!pip install pyzbar
!apt-get install -y zbar-tools

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  fonts-droid-fallback fonts-noto-mono fonts-urw-base35 ghostscript gsfonts
  imagemagick-6-common libdjvulibre-text libdjvulibre21 libfftw3-double3
  libgs9 libgs9-common libidn12 libijs-0.35 libjbig2dec0 libjxr-tools libjxr0
  liblqr-1-0 libmagickcore-6.q16-6 libmagickcore-6.q16-6-extra
  libmagickwand-6.q16-6 libv4l-0 libv4lconvert0 libwmflite-0.2-7 libzbar0
  poppler-data
Suggested packages:
  fonts-noto fonts-freefont-otf | fonts-freefont-ttf fonts-texgyre
  ghostscript-x libfftw3-bin libfftw3-dev inkscape poppler-utils
  fonts-japanese-mincho | fonts-ipafont-mincho fonts-japanese-gothic
  | fonts-ipafont-gothic fonts-arphic-ukai fonts-arphic-uming fonts-nanum
  zbarcam-gtk zbarcam-qt
The following NEW packages will be installed:
  fonts-droid-fallback fonts-noto-mono fonts-urw-base35 ghostscript gsfonts
  imagemagick-6-common libdjv